In [2]:
# primer_tools.py
"""
Minimal helper functions for primer QC

Requires:
    pip install primer3-py
"""

import primer3  # Python bindings to the Primer3 C core
from pathlib import Path
import numpy as np
from Bio.Seq import Seq


''' Check to find overhangs and primers to work for a Gibson assembly of the two. The overhangs of the AF is not really debatable.
    This script is to find overhangsd in the HDR arms. Check file "name convention for an illustration.
'''

#read files
leftHDR = Path("leftHDR.txt").read_text(encoding="utf-8")
rightHDR = Path("rightHDR.txt").read_text(encoding="utf-8")
leftAF = Path("leftAF.txt").read_text(encoding="utf-8")
rightAF = Path("rightAF.txt").read_text(encoding="utf-8")

def analyze_primer(seq, mv=50.0, dv=1.5, dntp=0.0, dna=250.0):
    seq = seq.upper().replace(" ", "")
    hairpin = primer3.calcHairpin(seq)
    homodimer = primer3.calcHomodimer(seq)

    return {
        "sequence": seq,
        "tm": primer3.calcTm(seq, mv_conc=mv, dv_conc=dv,
                             dntp_conc=dntp, dna_conc=dna),
        "hairpin_dg": hairpin.dg,           # attribute, not key
        "homodimer_dg": homodimer.dg,
    }

def analyze_pair(fwd: str, rev: str):
    """Return ΔG (kcal/mol) for one primer pair."""
    def _self_stats(seq):
        return {
            "hairpin_dg":  primer3.calcHairpin(seq).dg  / 1000.0,
            "homodimer_dg": primer3.calcHomodimer(seq).dg / 1000.0,
            "tm": primer3.calcTm(seq),
        }
    f = _self_stats(fwd)
    r = _self_stats(rev)
    hetero = primer3.calcHeterodimer(fwd, rev).dg / 1000.0
    return {"forward": f, "reverse": r, "heterodimer_dg": hetero}

def print_stats(stats):
    for side in ("forward", "reverse"):
        s = stats[side]
        print(f"{side:8}  Tm = {s['tm']:.1f} °C   "
              f"ΔGhairpin = {s['hairpin_dg']:.1f} kcal/mol   "
              f"ΔGhomodimer = {s['homodimer_dg']:.1f} kcal/mol ")

    print(f"Heterodimer ΔG = {stats['heterodimer_dg']:.1f} kcal/mol")

def check_pair(fwd, rev):
    hairpin_treshold = -3 #kcal/mol
    homodimer_treshold = -9
    heterodimer_threshold = -9

    stats = analyze_pair(fwd, rev)

    if stats['forward']['hairpin_dg'] < hairpin_treshold:
        return False
    if stats['forward']['homodimer_dg'] < homodimer_treshold:
        return False
    if stats['reverse']['hairpin_dg'] < hairpin_treshold:
        return False
    if stats['reverse']['homodimer_dg'] < homodimer_treshold:
        return False
    if stats['heterodimer_dg'] < heterodimer_threshold:
        return False
    
    return True

    

if __name__ == "__main__":
    FWD = "GGGGGGGGCCCCCCC"
    REV = "GATCGGGGGTtttaaaCGT"
    stats = analyze_pair(FWD, REV)

    if check_pair(FWD, REV):
        print('Yaaaay')
    else:
        print('Nooooo')
    print_stats(stats)
    


Nooooo
forward   Tm = 66.8 °C   ΔGhairpin = -4.8 kcal/mol   ΔGhomodimer = -20.6 kcal/mol 
reverse   Tm = 55.5 °C   ΔGhairpin = 0.0 kcal/mol   ΔGhomodimer = -3.2 kcal/mol 
Heterodimer ΔG = -6.2 kcal/mol


/Users/jonfreeman/.pyenv/versions/3.10.13/envs/jupyter/lib/python3.10/site-packages/primer3/bindings.py:135: UserWarning: Function deprecated please use "calc_hairpin" instead
  return THERMO_ANALYSIS.calcHairpin(seq, output_structure).check_exc()
/Users/jonfreeman/.pyenv/versions/3.10.13/envs/jupyter/lib/python3.10/site-packages/primer3/bindings.py:217: UserWarning: Function deprecated please use "calc_homodimer" instead
  return THERMO_ANALYSIS.calcHomodimer(seq, output_structure).check_exc()
/Users/jonfreeman/.pyenv/versions/3.10.13/envs/jupyter/lib/python3.10/site-packages/primer3/bindings.py:489: UserWarning: Function deprecated please use "calc_tm" instead
  return THERMO_ANALYSIS.calcTm(seq)
/Users/jonfreeman/.pyenv/versions/3.10.13/envs/jupyter/lib/python3.10/site-packages/primer3/bindings.py:305: UserWarning: Function deprecated please use "calc_heterodimer" instead
  return THERMO_ANALYSIS.calcHeterodimer(


In [3]:
def get_snippet(
    seq: str,
    *,
    start: int,
    length: int,
    origin: str = "head",
) -> str:
    """
    Extract a subsequence of given length from `seq`.

    Parameters
    ----------
    seq : str
        Full sequence (upper/lower case OK).
    start : int
        Offset where to begin (0-based).  Interpretation depends on `origin`.
    length : int
        Number of characters to return.
    origin : {"head", "tail"}
        • "head" → start is counted from 5' end (index 0).  
        • "tail" → start is counted from 3' end (index len(seq)-1).

    Returns
    -------
    str
        Requested subsequence. Raises ValueError if requested range
        would go outside the sequence bounds.
    """
    seq = seq.upper()
    n = len(seq)

    if length <= 0:
        raise ValueError("length must be positive")

    if origin not in {"head", "tail"}:
        raise ValueError("origin must be 'head' or 'tail'")

    if origin == "head":
        i0 = start
    else:  # origin == "tail"
        i0 = n - start - length

    i1 = i0 + length

    if i0 < 0 or i1 > n:
        raise ValueError(
            f"requested range [{i0}:{i1}] lies outside sequence of length {n}"
        )

    return seq[i0:i1]

#check
a = "AGGTAGCATGACTGTTTAGTTTA"
b = "GGTAGCATGGGATACAGTATATT"
print(get_snippet(a,start=3,length=4,origin="tail"))

TAGT


In [23]:
class Primers:
    def __init__(self, oh_leftHDR, oh_rightHDR, oh_leftAF, oh_rightAF,
                 HDRfw, HDRrv, AFfw, AFrv, hdr_starts=(0, 0)):
        self.oh_leftHDR  = oh_leftHDR.upper()
        self.oh_rightHDR = oh_rightHDR.upper()
        self.oh_leftAF   = oh_leftAF.upper()
        self.oh_rightAF  = oh_rightAF.upper()
        self.HDRfw = HDRfw.upper()
        self.HDRrv = HDRrv.upper()
        self.AFfw  = AFfw
        self.AFrv  = AFrv
        self.hdr_starts = hdr_starts

        pairs = [
            (self.oh_leftHDR, self.oh_rightHDR),  # HDR-only tails
            (self.HDRfw,      self.HDRrv),        # HDR Gibson pair
            (self.AFfw,       self.AFrv),         # AF Gibson pair
        ]

        dGs = []
        for fwd, rev in pairs:
            stats = analyze_pair(fwd, rev)
            dGs += [
                stats["forward"]["hairpin_dg"],
                stats["forward"]["homodimer_dg"],
                stats["reverse"]["hairpin_dg"],
                stats["reverse"]["homodimer_dg"],
                stats["heterodimer_dg"],
            ]

        self.dg_error = (np.array(dGs)**2).sum()
    
    def report(self):
        print(f"\n🔬 Primer Pair Report (HDR starts: {self.hdr_starts})\n")
        pairs = [
            ("HDR-only", self.oh_leftHDR, self.oh_rightHDR),
            ("HDR",      self.HDRfw,      self.HDRrv),
            ("AF",       self.AFfw,       self.AFrv),
        ]

        for label, fwd, rev in pairs:
            stats = analyze_pair(fwd, rev)
            print(f"🧪 {label}")
            print(f"  FWD (len={len(fwd)}): Tm={stats['forward']['tm']:.1f}°C | "
                  f"ΔG_hairpin={stats['forward']['hairpin_dg']:.2f} | "
                  f"ΔG_homodimer={stats['forward']['homodimer_dg']:.2f}")
            print(f"  REV (len={len(rev)}): Tm={stats['reverse']['tm']:.1f}°C | "
                  f"ΔG_hairpin={stats['reverse']['hairpin_dg']:.2f} | "
                  f"ΔG_homodimer={stats['reverse']['homodimer_dg']:.2f}")
            print(f"  Heterodimer ΔG = {stats['heterodimer_dg']:.2f} kcal/mol\n")
    
    def show_primers(self):
        """Pretty-print all eight primer sequences with length and Tm."""
        primers = {
            "oh_leftHDR":  self.oh_leftHDR,
            "oh_rightHDR": self.oh_rightHDR,
            "oh_leftAF":   self.oh_leftAF,
            "oh_rightAF":  self.oh_rightAF,
            "HDRfw":       self.HDRfw,
            "HDRrv":       self.HDRrv,
            "AFfw":        self.AFfw,
            "AFrv":        self.AFrv,
        }

        print("\n🧬 Primer list")
        print("┌────────────┬─────────────────────────────────────────┬───────┬──────┐")
        print("│ Name       │ Sequence                                │  Len  │  Tm  │")
        print("├────────────┼─────────────────────────────────────────┼───────┼──────┤")

        for name, seq in primers.items():
            tm_val = calcTm(seq)                           # uses primer3 (full primer)
            print(f"│ {name:<10} │ {seq:<41} │ {len(seq):>4} │ {tm_val:5.1f} │")

        print("└────────────┴─────────────────────────────────────────┴───────┴──────┘\n"


    def __repr__(self):
        return f"Primers(RMSE ΔG={self.rmse_dG:.2f} kcal/mol, HDR starts={self.hdr_starts})"
    

In [24]:
'''Set parameters'''
#minimum length of overhang
min_overhang = 17
#maximum length of overhang
max_overhang = 23
#primer range
range_overhang = range(min_overhang, max_overhang)

#how many bases of the HDR arms can be skipped
latest_start_HDR = 30

#list for the good stuff
possible_primers = []


##### MAIN LOOP #####
#loops over possible left and right starts
for start_left in range(latest_start_HDR):
    for start_right in range(latest_start_HDR):

        #loops over possible lengths of overhangs
        for oh_leftHDR in range_overhang:
            for oh_rightHDR in range_overhang:
                for oh_leftAF in range_overhang:
                    for oh_rightAF in range_overhang:

                        #define sequences (see file "namin convention.pdf" for orientation)
                        overhang_leftHDR = get_snippet(leftHDR, start=start_left, length=oh_leftHDR, origin="head")
                        overhang_rightHDR = get_snippet(rightHDR, start=start_right, length=oh_rightHDR, origin="tail")

                        overhang_leftAF = get_snippet(leftAF, start=0, length=oh_rightAF, origin="head")
                        overhang_rightAF = get_snippet(leftAF, start=0, length=oh_rightAF, origin="head")

                        HDRfw = overhang_rightAF + overhang_leftHDR
                        HDRrv = overhang_rightHDR + overhang_leftAF
                        
                        AFfw  = str(Seq(HDRrv).reverse_complement())
                        AFrv  = str(Seq(HDRfw).reverse_complement())


                        if check_pair(HDRfw, HDRrv) and check_pair(AFfw, AFrv) and check_pair(overhang_leftHDR, overhang_rightHDR):
                                print("I GOT AN OPTION!")
                                new_primers = Primers(overhang_leftHDR, overhang_rightHDR, overhang_leftAF, overhang_rightAF, HDRfw, HDRrv, AFfw, AFrv, (start_left, start_right))
                                possible_primers.append(new_primers)
                                
                                
                       
                    


I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTION!
I GOT AN OPTIO

KeyboardInterrupt: 

In [25]:
print(f'Length of possible primers list: {len(possible_primers)}')

possible_primers[10].report()

the_primers = possible_primers[10]



Length of possible primers list: 270

🔬 Primer Pair Report (HDR starts: (0, 0))

🧪 HDR-only
  FWD (len=17): Tm=60.0°C | ΔG_hairpin=-0.87 | ΔG_homodimer=-6.06
  REV (len=17): Tm=55.4°C | ΔG_hairpin=0.00 | ΔG_homodimer=-6.74
  Heterodimer ΔG = -3.44 kcal/mol

🧪 HDR
  FWD (len=34): Tm=74.2°C | ΔG_hairpin=-1.47 | ΔG_homodimer=-6.06
  REV (len=34): Tm=70.6°C | ΔG_hairpin=0.13 | ΔG_homodimer=-6.74
  Heterodimer ΔG = -3.76 kcal/mol

🧪 AF
  FWD (len=34): Tm=70.6°C | ΔG_hairpin=-0.83 | ΔG_homodimer=-5.96
  REV (len=34): Tm=74.2°C | ΔG_hairpin=-1.07 | ΔG_homodimer=-5.64
  Heterodimer ΔG = -4.01 kcal/mol

